<h2>개인 구글 드라이브와 colab 연동</h2>

In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

"""
처음 부분은 불용어를 제거한 경우에 대한 실습 결과입니다.(7점을 위한 것)

두 번째 부분은 성능을 추가하기 위해서 한 글자 혹은 두 글자인 단어인 경우와 일부 특수문자를 제거한 경우를 포함시켜 성능을 향상 시킨 실습 결과입니다.(10점을 위한 것)
"""

Mounted at /gdrive


'\n처음 부분은 불용어를 제거한 경우에 대한 실습 결과입니다.(7점을 위한 것)\n\n두 번째 부분은 성능을 추가하기 위해서 한 글자 혹은 두 글자인 단어인 경우와 일부 특수문자를 제거한 경우를 포함시켜 성능을 향상 시킨 실습 결과입니다.(10점을 위한 것)\n'

<h2>"SMSSpamCollection" 데이터를 읽고 문장과 정답을 분리하여 각 리스트에 저장</h2>

<pre>
<b>1. 데이터의 형태(SMSSpamCollection)</b>
  라벨(스팸 또는 햄) \t(tab) 문장
  
  위와 같은 형태로 저장되어 있음
  
  예시)
    ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    spam\tCustomer service annoncement. You have a New Years delivery waiting for you. Please call 07046744435 now to arrange delivery
    ...
  
  따라서 입력 데이터를 읽고 \t을 기준으로 입력 문장을 분리한 후에 문장과 라벨을 각각 x_data, y_data 리스트에 저장
  
<b>2. 입력 데이터 전체를 사용하지 않고 100개만 추출해서 사용</b>

<b>3. x_data, y_data 형태</b>
  x_data = [ 문장1, 문장2, 문장3, ... 문장100]
  y_data = [ 문장1의 라벨, 문장2의 라벨, 문장3의 라벨, ... 문장100의 라벨]
</pre>

In [ ]:
import numpy as np

file_path = "/gdrive/MyDrive/Colab/Week04/SMSSpamCollection.dat"
"""
svm은 binary classification에서 가장 좋은 효율
만약 여러 개라면 A와 나머지, B와 나머지 같은 방식을 이용
"""

# 파일 읽기
x_data, y_data = [], []
with open(file_path,'r',encoding='utf8') as inFile:
  lines = inFile.readlines()

lines = lines[:100]  #100개만 잘라서 라인 단위로 사용!
for line in lines:
  line = line.strip().split('\t')#tab 단위로 분리. strip 앞뒤 화이트 스페이스 제거하는 함수
  sentence, label = line[1], line[0]#sentence와 label(정답)을 분리
  x_data.append(sentence)#sentence는 x 데이터
  y_data.append(label)#정답인 label은 y 데이터

print("x_data의 개수 : " + str(len(x_data)))
print("y_data의 개수 : " + str(len(y_data)))

x_data의 개수 : 100
y_data의 개수 : 100


<h2>Tokenizer 라이브러리를 사용하여 입력 문장을 index로 치환</h2>

<pre>
<b>1. tokenizer.fit_on_texts(data) 함수를 이용하여 각 단어를 index로 치환하기 위한 딕셔너리 생성</b>
   생성된 딕셔너리는 tokenizer 객체 안에 저장됨
  
  tokenizer.fit_on_texts(data)
  args
    data : 문자열 element를 가지고 있는 리스트
  return
    X
    
  딕셔너리 예시)
    {'to': 1, 'i': 2, 'you': 3, 'a': 4, 'the': 5, 'and': 6, 'for': 7 ... }
    
<b>2. tokenizer.texts_to_sequences(data) 함수를 이용하여 문장 안에 있는 단어들을 index로 치환</b>

  tokenizer.texts_to_sequence(data)
  args
    data : 문자열 element를 가지고 있는 리스트
  return :
    indexing 된 리스트
    
  indexing 예시)
    x_data indexing 하기 전 : Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    x_data indexing 하기 후 : [38, 93, 239, 240, 241, 242, 53, 11, 243, 72, 94, 244, 245, 126, 246, 247, 73, 74, 248, 127]
    y_data indexing 하기 전 : ham
    y_data indexing 하기 후 : 1
</pre>

In [ ]:
from keras.preprocessing.text import Tokenizer
import nltk

tokenizer = Tokenizer()#스페이스 단위로 Token으로 취급


# spam, ham 라벨을 대응하는 index로 치환하기위한 딕셔너리
label2index_dict = {'spam':0, 'ham':1}

# indexing 한 데이터를 넣을 리스트 선언
indexing_x_data, indexing_y_data = [], []
#숫자로 받을 리스트

for label in y_data:
  indexing_y_data.append(label2index_dict[label])
#label을 딕셔너리를 통해서 리스트에 대입해서 변환

# x_data를 사용하여 딕셔너리 생성
tokenizer.fit_on_texts(x_data)
#문장인 x_data는 단어가 굉장히 많은데 이 함수를 통해서 unique하게 sort해서 중복을 제거하고 번호로 딕셔너리를 통해 indexing하는 것을 vocabularay라고 함!
#이 vocabularay의 원소가 한 개당 1차원으로 svm은 인식
nltk.download('stopwords')
stopwordlist = nltk.corpus.stopwords.words('english')

i=0
flag=False
for sentence in x_data:
  sentenceList = sentence.split()
  newSentenceList=[]
  for word in sentenceList:
    if word.lower() in stopwordlist: #불용어 체크해서 해당 단어가 불용어인 경우에는 제외
      continue
    else:
      newSentenceList.append(word)
      flag=True
  if flag==True:
    flag=False
    newSentence = ' '.join(map(str, newSentenceList))
    x_data[i]=newSentence
  i+=1
# x_data에 있는 각 문장의 단어들을 대응하는 index로 치환하고 그 결과값을 indexing_x_data에 저장
indexing_x_data = tokenizer.texts_to_sequences(x_data)
#만든 딕셔너리를 활용해서 숫자들로 인덱싱해서 리스트로 변환

print("x_data indexing 하기 전 : " + str(x_data[0]))
print("x_data indexing 하기 후 : " + str(indexing_x_data[0]))
print("y_data indexing 하기 전 : " + str(y_data[0]))
print("y_data indexing 하기 후 : " + str(indexing_y_data[0]))

x_data indexing 하기 전 : Go jurong point, crazy.. Available bugis n great world la e buffet... Cine got amore wat...
x_data indexing 하기 후 : [38, 239, 240, 241, 242, 243, 72, 94, 244, 245, 126, 246, 247, 74, 248, 127]
y_data indexing 하기 전 : ham
y_data indexing 하기 후 : 1


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h2>SVM 모델 학습</h2>

<pre>
<b>1. 데이터의 문장 길이를 고정된 길이로 변환</b>
  예제 코드에서는 60으로 맞추어 변환
  
  문장 길이가 60 초과인 경우 뒷 부분 제거
  문장 길이가 60 미만인 경우 나머지 부분을 0으로 채움
  
  예시)
    문장 길이를 5로 맞추고자 할 경우
    
    문장 길이가 5보다 큰 경우
    문장 : 나는 어제 집에서 저녁으로 김치찌개를 먹었다, indexing_문장 : 38, 93, 239, 240, 241, 242
    38, 93, 239, 240, 241, 242 -> 38, 93, 239, 240, 241
    
    문장 길이가 5보다 작은 경우
    문장 : 나는 김치찌개를 좋아해, indexing_문장 : 74, 248, 127
    74, 248, 127 -> 74, 248, 127, 0, 0
    
<b>2. 입력 데이터를 9 대 1 비율로 나누어 학습, 평가에 사용</b>
  train_x = [ 문장1, 문장2, 문장3, ... 문장90]
  train_y = [ 문장1의 라벨, 문장2의 라벨, 문장3의 라벨, ... 문장90의 라벨]
  test_x = [ 문장91, 문장92, 문장93, ... 문장100]
  test_y = [ 문장91의 라벨, 문장92의 라벨, 문장93의 라벨, ... 문장100의 라벨]
  
<b>3. svm.fit(x, y) 함수를 사용하여 SVM 모델 학습</b>
  svm.fit(x, y)
  args
    x : indexing 된 문장들이 있는 리스트
    y : x의 각 문장에 대응하는 라벨이 있는 리스트
  return :
    X
</pre>

In [ ]:
from sklearn.svm import SVC#SVClassification만 사용!

# 문장의 길이를 max_length으로 맞춰 변환
max_length = 60
for index in range(len(indexing_x_data)):#모든 인덱스에 대해서 반복! 모든 데이터를 같은 길이로 보정
#테스트 데이터가 얼마나 긴 게 들어올 지는 모르나 한정된 길이만 기준으로 checking 해야함
  length = len(indexing_x_data[index])

  if(length > max_length):#길이가 넘어가면 자름
    indexing_x_data[index] = indexing_x_data[index][:max_length]
  elif(length < max_length):#아니면 임의의 값인 0으로 추가 이를 padding이라고 함
    indexing_x_data[index] = indexing_x_data[index] + [0]*(max_length-length)


# 전체 데이터를 9:1의 비율로 나누어 학습 및 평가 데이터로 사용
number_of_train = int(len(indexing_x_data)*0.9)

#train data랑 test data 분리
train_x = indexing_x_data[:number_of_train]
train_y = indexing_y_data[:number_of_train]
test_x = indexing_x_data[number_of_train:]
test_y = indexing_y_data[number_of_train:]

print("train_x의 개수 : " + str(len(train_x)))
print("train_y의 개수 : " + str(len(train_y)))
print("test_x의 개수 : " + str(len(test_x)))
print("test_y의 개수 : " + str(len(test_y)))

svm = SVC(kernel='linear', C=1e10)#svm에 kernel function을 linear로 설정하고 C는 margin 내부의 Slack을 위한 변수로 error 허용 SVC 객체 담기
#차원이 굉장히 큰 문제인 경우 linear kernel function 사용! -> 여기서는 단어 한 개가 1차원이라서 차원이 굉장히 크다!
#차원이 작거나 100차원이내의 kernel이면 RBF나 다른 kernel function이 성능이 좋다.
svm.fit(train_x, train_y)#학습 시작!

train_x의 개수 : 90
train_y의 개수 : 90
test_x의 개수 : 10
test_y의 개수 : 10


SVC(C=10000000000.0, kernel='linear')

<h2>SVM 모델을 이용한 평가</h2>

<pre>
<b>1. svm.predict(data) 함수를 사용하여 SVM 모델을 이용하여 평가</b>
  
  svm.predict(data)
  args
    data : indexing 된 문장들이 있는 리스트
  return :
    입력 문장들에 대한 모델의 출력 라벨 리스트
    
<b>2. 성능 측정</b>
  정답 라벨과 모델의 출력 라벨을 비교하여 성능 측정
  
<b>3. tokenizer.sequences_to_texts(data) 함수를 이용하여 indexing 된 데이터를 단어로 치환</b>

  tokenizer.sequences_to_texts(data)
  args
    data : indexing 된 리스트
  return :
    단어로 치환된 리스트
    
  예시)
    [38, 93, 239, 240, 241, 242, 53, 11, 243, 72, 94, 244, 245, 126, 246, 247, 73, 74, 248, 127] -> Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    
<b>4. 입력 문장에 대한 모델의 출력과 정답 출력</b>

</pre>

In [ ]:
predict = svm.predict(test_x)#test_x는 test할 데이터로 예측하고 그 결과를 predict 변수에 담기!(리스트 단위 반환)

correct_count = 0
for index in range(len(predict)):
  if(test_y[index] == predict[index]):#예측으로 나온 값이 정답과 비교
    correct_count += 1

accuracy = 100.0*correct_count/len(test_y)


print("Accuracy: " + str(accuracy))

index2label = {0:"spam", 1:"ham"}
#다시 index를 label로 변환
test_x_word = tokenizer.sequences_to_texts(test_x)
#다시 index를 sentence로 변환
for index in range(len(test_x_word)):
  print()
  print("문장 : ", test_x_word[index])
  print("정답 : ", index2label[test_y[index]])
  print("모델 출력 : ", index2label[predict[index]])

Accuracy: 60.0

문장 :  yeah do don‘t stand close tho you‘ll catch something
정답 :  ham
모델 출력 :  ham

문장 :  sorry pain ok meet another night spent late afternoon casualty means done stuff42moro includes time sheets that sorry
정답 :  ham
모델 출력 :  ham

문장 :  smile pleasure smile pain smile trouble pours like rain smile sum1 hurts u smile becoz someone still loves see u smiling
정답 :  ham
모델 출력 :  spam

문장 :  please call customer service representative 0800 169 6031 10am 9pm guaranteed ￡1000 cash ￡5000 prize
정답 :  spam
모델 출력 :  ham

문장 :  havent planning buy later check already lido got 530 show e afternoon u finish work already
정답 :  ham
모델 출력 :  spam

문장 :  free ringtone waiting collected simply text password mix 85069 verify get usher britney fml po box 5249 mk17 92h 450ppw 16
정답 :  spam
모델 출력 :  spam

문장 :  watching telugu movie wat abt u
정답 :  ham
모델 출력 :  ham

문장 :  see finish loads loans pay
정답 :  ham
모델 출력 :  ham

문장 :  hi wk ok hols now yes bit run forgot hairdressers appointment four

In [ ]:
import nltk

nltk.download('stopwords')
"""
불용어 리스트를 참고해서 입력 문장에서 불용어를 제거한 SVM 기반 스팸 메일 필터를 구현하기!
불용어는 사용할 필요 없는 말!(a, the, 은/는/이/가 같은 리스트를 stopwordlist인 불용어리스트라고 한다!)
즉, 불용어는 정보 검색에 대해서는 필요없으니 여기서 단어 1개가 1개의 차원이니까
불용어를 없앤다는 것은 곧 차원을 줄인다는 것!

feature를 추가한다면
스팸을 조사해서 쓸데없는 말을 제거하는 것말고 중요한 단어를 선택하는 방법인 카이스퀘어통계량을 이용 가능
한 단어가 아닌 두 단어를 기준으로 1차원을 응용해서 차원을 줄이기
mutual impormation같은 상호 정보량을 이용해서 차원을 줄이기

"""
print('영어 불용어 갯수 : ', len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:10])

영어 불용어 갯수 :  179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
"""
여기서부터는 성능 개선을 위한 Feature 추가입니다.
"""

import numpy as np

file_path = "/gdrive/MyDrive/Colab/Week04/SMSSpamCollection.dat"
"""
svm은 binary classification에서 가장 좋은 효율
만약 여러 개라면 A와 나머지, B와 나머지 같은 방식을 이용
"""

# 파일 읽기
x_data, y_data = [], []
with open(file_path,'r',encoding='utf8') as inFile:
  lines = inFile.readlines()

lines = lines[:100]  #100개만 잘라서 라인 단위로 사용!
for line in lines:
  line = line.strip().split('\t')#tab 단위로 분리. strip 앞뒤 화이트 스페이스 제거하는 함수
  sentence, label = line[1], line[0]#sentence와 label(정답)을 분리
  x_data.append(sentence)#sentence는 x 데이터
  y_data.append(label)#정답인 label은 y 데이터

print("x_data의 개수 : " + str(len(x_data)))
print("y_data의 개수 : " + str(len(y_data)))

x_data의 개수 : 100
y_data의 개수 : 100


In [ ]:
from keras.preprocessing.text import Tokenizer
import nltk

tokenizer = Tokenizer()#스페이스 단위로 Token으로 취급


# spam, ham 라벨을 대응하는 index로 치환하기위한 딕셔너리
label2index_dict = {'spam':0, 'ham':1}

# indexing 한 데이터를 넣을 리스트 선언
indexing_x_data, indexing_y_data = [], []
#숫자로 받을 리스트

for label in y_data:
  indexing_y_data.append(label2index_dict[label])
#label을 딕셔너리를 통해서 리스트에 대입해서 변환

# x_data를 사용하여 딕셔너리 생성
tokenizer.fit_on_texts(x_data)
#문장인 x_data는 단어가 굉장히 많은데 이 함수를 통해서 unique하게 sort해서 중복을 제거하고 번호로 딕셔너리를 통해 indexing하는 것을 vocabularay라고 함!
#이 vocabularay의 원소가 한 개당 1차원으로 svm은 인식
nltk.download('stopwords')
stopwordlist = nltk.corpus.stopwords.words('english')

i=0
flag=False
for sentence in x_data:
  sentenceList = sentence.split()
  newSentenceList=[]
  for word in sentenceList:
    if (len(word) == 1):# 한 글자는 상대적으로 큰 의미가 없으니까 학습에서 제거한다. 이렇게 하면 포함되는 word수가 적어지니 차원의 수가 적어진다.
        continue
    if (word.lower() in stopwordlist):
      #불용어 체크해서 해당 단어가 불용어인 경우에는 제외
      continue
    else:
      word = word.replace('.', '').replace(',', '').replace('#','').replace('!','').replace('@','')
      #일부 많이 사용되는 특수 문자는 상대적으로 큰 의미가 없고 오히려 같은 단어이지만 일부 특수문자로 구별되는 경우를 방지해 차원의 수를 줄인다.
      #즉, apple과 apple!같은 경우 같은 단어이지만 !라는 특수문자로 인해서 다른 단어로 분류되어 차원의 수를 높이는 경우를 방지하기 위함이다.
      newSentenceList.append(word)
      flag=True
  if flag==True:
    flag=False
    newSentence = ' '.join(map(str, newSentenceList))
    x_data[i]=newSentence
  i+=1
# x_data에 있는 각 문장의 단어들을 대응하는 index로 치환하고 그 결과값을 indexing_x_data에 저장
indexing_x_data = tokenizer.texts_to_sequences(x_data)
#만든 딕셔너리를 활용해서 숫자들로 인덱싱해서 리스트로 변환

print("x_data indexing 하기 전 : " + str(x_data[0]))
print("x_data indexing 하기 후 : " + str(indexing_x_data[0]))
print("y_data indexing 하기 전 : " + str(y_data[0]))
print("y_data indexing 하기 후 : " + str(indexing_y_data[0]))

x_data indexing 하기 전 : Go jurong point crazy Available bugis great world la buffet Cine got amore wat
x_data indexing 하기 후 : [38, 239, 240, 241, 242, 243, 94, 244, 245, 246, 247, 74, 248, 127]
y_data indexing 하기 전 : ham
y_data indexing 하기 후 : 1


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.svm import SVC#SVClassification만 사용!

# 문장의 길이를 max_length으로 맞춰 변환
max_length = 60
for index in range(len(indexing_x_data)):#모든 인덱스에 대해서 반복! 모든 데이터를 같은 길이로 보정
#테스트 데이터가 얼마나 긴 게 들어올 지는 모르나 한정된 길이만 기준으로 checking 해야함
  length = len(indexing_x_data[index])

  if(length > max_length):#길이가 넘어가면 자름
    indexing_x_data[index] = indexing_x_data[index][:max_length]
  elif(length < max_length):#아니면 임의의 값인 0으로 추가 이를 padding이라고 함
    indexing_x_data[index] = indexing_x_data[index] + [0]*(max_length-length)


# 전체 데이터를 9:1의 비율로 나누어 학습 및 평가 데이터로 사용
number_of_train = int(len(indexing_x_data)*0.9)

#train data랑 test data 분리
train_x = indexing_x_data[:number_of_train]
train_y = indexing_y_data[:number_of_train]
test_x = indexing_x_data[number_of_train:]
test_y = indexing_y_data[number_of_train:]

print("train_x의 개수 : " + str(len(train_x)))
print("train_y의 개수 : " + str(len(train_y)))
print("test_x의 개수 : " + str(len(test_x)))
print("test_y의 개수 : " + str(len(test_y)))

svm = SVC(kernel='linear', C=1e10)#svm에 kernel function을 linear로 설정하고 C는 margin 내부의 Slack을 위한 변수로 error 허용 SVC 객체 담기
#차원이 굉장히 큰 문제인 경우 linear kernel function 사용! -> 여기서는 단어 한 개가 1차원이라서 차원이 굉장히 크다!
#차원이 작거나 100차원이내의 kernel이면 RBF나 다른 kernel function이 성능이 좋다.
svm.fit(train_x, train_y)#학습 시작!

train_x의 개수 : 90
train_y의 개수 : 90
test_x의 개수 : 10
test_y의 개수 : 10


SVC(C=10000000000.0, kernel='linear')

In [ ]:
predict = svm.predict(test_x)#test_x는 test할 데이터로 예측하고 그 결과를 predict 변수에 담기!(리스트 단위 반환)

correct_count = 0
for index in range(len(predict)):
  if(test_y[index] == predict[index]):#예측으로 나온 값이 정답과 비교
    correct_count += 1

accuracy = 100.0*correct_count/len(test_y)


print("Accuracy: " + str(accuracy))

index2label = {0:"spam", 1:"ham"}
#다시 index를 label로 변환
test_x_word = tokenizer.sequences_to_texts(test_x)
#다시 index를 sentence로 변환
for index in range(len(test_x_word)):
  print()
  print("문장 : ", test_x_word[index])
  print("정답 : ", index2label[test_y[index]])
  print("모델 출력 : ", index2label[predict[index]])

  """
  기존 불용어를 제외하고
  추가로 한 글자인 단어를 제외하고
  최종적으로 남은 단어들의 일부 의미없는 특수문자들을 제거한 결과 성능이 향상되었다.
  """

Accuracy: 90.0

문장 :  yeah do don‘t stand close tho you‘ll catch something
정답 :  ham
모델 출력 :  ham

문장 :  sorry pain ok meet another night spent late afternoon casualty means done stuff42moro includes time sheets that sorry
정답 :  ham
모델 출력 :  ham

문장 :  smile pleasure smile pain smile trouble pours like rain smile sum1 hurts smile becoz someone still loves see smiling
정답 :  ham
모델 출력 :  spam

문장 :  please call customer service representative 0800 169 6031 10am 9pm guaranteed ￡1000 cash ￡5000 prize
정답 :  spam
모델 출력 :  spam

문장 :  havent planning buy later check already lido got 530 show afternoon finish work already
정답 :  ham
모델 출력 :  ham

문장 :  free ringtone waiting collected simply text password mix 85069 verify get usher britney fml po box 5249 mk17 92h 450ppw 16
정답 :  spam
모델 출력 :  spam

문장 :  watching telugu abt u
정답 :  ham
모델 출력 :  ham

문장 :  see finish loads loans pay
정답 :  ham
모델 출력 :  ham

문장 :  hi wk ok hols now yes bit run forgot hairdressers appointment four need get home sho